In [1]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV,LassoCV,ElasticNetCV
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle as pkl

<h1>Load Dataset and feature Matrix</h1>

In [20]:
# images = pd.read_csv('../../Datasets/Assignment 3/Ages.csv')
images = pd.read_csv('../../Assignment 1/task1/Data.csv')

In [21]:
# file_name = '../../Datasets/Assignment 3/Compressed_images/Compressed_data_age_1'
file_name = '../../Assignment 1/task1/Compressed_data_1'
with open(file_name, 'rb') as handle:
    feature_matrix = pkl.load(handle)
print(feature_matrix.shape)

(61, 3072)


In [23]:
X_train,X_test,y_train,y_test = train_test_split(feature_matrix,images['age'],test_size=0.2)

In [24]:
X_train.shape,X_test.shape

((48, 3072), (13, 3072))

In [25]:
reg = SGDRegressor(random_state=10,early_stopping=True)

In [35]:
params = {'alpha':[0.001,0.01,0.1],'penalty':['l2','l1','elasticnet',None],'l1_ratio':[0.1,0.15,0.18],'max_iter':[1000],'loss':['huber']}

In [36]:
searcher = GridSearchCV(reg,param_grid=params,cv=10,scoring='r2',n_jobs=-1)

In [37]:
searcher.fit(X_train,y_train)

GridSearchCV(cv=10,
             estimator=SGDRegressor(alpha=0.001, early_stopping=True,
                                    l1_ratio=0, loss='huber', random_state=10),
             n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1],
                         'l1_ratio': [0.1, 0.15, 0.18], 'loss': ['huber'],
                         'max_iter': [1000],
                         'penalty': ['l2', 'l1', 'elasticnet', None]},
             scoring='r2')

In [45]:
searcher_rigid = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1],cv=5).fit(X_train,y_train)

In [48]:
searcher_lasso = LassoCV(alphas=[1e-3, 1e-2, 1e-1, 1],cv=5, max_iter=2000,random_state=32).fit(X_train, y_train)

In [51]:
searcher_elas = ElasticNetCV(alphas=[1e-3, 1e-2, 1e-1, 1],cv=5,max_iter=3000, random_state=32).fit(X_train, y_train)

c:\Python\Python38\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.03802621480033362, tolerance: 0.02994791666666667
  model = cd_fast.enet_coordinate_descent(


In [38]:
# y_pred = searcher.predict(X_test)
reg = searcher.best_estimator_
print(searcher.best_params_)

{'alpha': 0.001, 'l1_ratio': 0.1, 'loss': 'huber', 'max_iter': 1000, 'penalty': 'l2'}


In [52]:
y_pred_grid = reg.predict(X_test)
y_pred_rig = searcher_rigid.predict(X_test)
y_pred_lass = searcher_lasso.predict(X_test)
y_pred_elas = searcher_elas.predict(X_test)


In [53]:
mse_test_grid = MSE(y_test,y_pred_grid)
r2_test_grid = r2_score(y_test,y_pred_grid)

mse_test_rig = MSE(y_test,y_pred_rig)
r2_test_rig = r2_score(y_test,y_pred_rig)

mse_test_lass = MSE(y_test,y_pred_lass)
r2_test_lass = r2_score(y_test,y_pred_lass)

mse_test_elas = MSE(y_test,y_pred_elas)
r2_test_elas = r2_score(y_test,y_pred_elas)

In [43]:
# model_file = 'SGF_Linear_Reg_Images_UTKFace.sav'
model_file = 'SGF_Linear_Reg_Images_my_data.sav'
pkl.dump(reg,open(model_file,'wb'))

In [44]:
# model_file = 'SGF_Linear_Reg_Images_UTKFace.sav'
model_file = 'SGF_Linear_Reg_Images_my_data.sav'
reg = pkl.load(open(model_file,'rb'))

<h1 style='text-align:center'>Model evaluation</h1>

Linear Regression 
<br>
Mean squared error 0.5069245345783313
<br>
Root mean squared error: 0.7119863303310895


In [54]:
print(f'Parameters: {reg.get_params()}')
print(f'Coefficient: {reg.coef_}')
print(f'Intercept: {reg.intercept_}')
print(f'Mean squared error test of grid: {mse_test_grid}')
print(f'R2 score of grid: {r2_test_grid}')
print(f'Mean squared error test of grid: {mse_test_rig}')
print(f'R2 score of grid: {r2_test_rig}')
print(f'Mean squared error test of grid: {mse_test_lass}')
print(f'R2 score of grid: {r2_test_lass}')
print(f'Mean squared error test of grid: {mse_test_elas}')
print(f'R2 score of grid: {r2_test_elas}')

Parameters: {'alpha': 0.001, 'average': False, 'early_stopping': True, 'epsilon': 0.1, 'eta0': 0.01, 'fit_intercept': True, 'l1_ratio': 0.1, 'learning_rate': 'invscaling', 'loss': 'huber', 'max_iter': 1000, 'n_iter_no_change': 5, 'penalty': 'l2', 'power_t': 0.25, 'random_state': 10, 'shuffle': True, 'tol': 0.001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
Coefficient: [0.03361199 0.04068638 0.03755518 ... 0.02035432 0.01855502 0.00662561]
Intercept: [0.09529039]
Mean squared error test of grid: 1.989315988345754
R2 score of grid: 0.6638055979695676
Mean squared error test of grid: 0.692535855720997
R2 score of grid: 0.8829614403831515
Mean squared error test of grid: 0.8471563273318375
R2 score of grid: 0.8568305806809194
Mean squared error test of grid: 0.6844173365321287
R2 score of grid: 0.8843334701260703
